In [2]:
print("ok")

ok


In [3]:
pip install -U langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

Note: you may need to restart the kernel to use updated packages.Collecting langchain-community
  Using cached langchain_community-0.3.14-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached pymupdf-1.25.1-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
  Using cached tiktoken-0.8.0-cp310-cp310-win_amd64.whl.metadata (6.8 kB)
  Using cached langchain_ollama-0.2.2-py3-none-any.whl.metadata (1.9 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.36-cp310-cp310-win_amd64.whl.metadata (9.9 kB)
  Using cached aiohttp-3.11.11-cp310-cp310-win_amd64.whl.metadata (8.0 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.

In [4]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

True

In [5]:
os.environ['LANGCHAIN_PROJECT']

'chat_pdf'

In [8]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(r"data\1. Analysis of Actual Fitness Supplement.pdf")

docs = loader.load()

In [10]:
doc=docs[0]

In [21]:
print(doc.page_content)

Citation: Espeño, P.R.; Ong, A.K.S.;
German, J.D.; Gumasing, M.J.J.; Casas,
E.S. Analysis of Actual Fitness
Supplement Consumption among
Health and Fitness Enthusiasts. Foods
2024, 13, 1424. https://doi.org/
10.3390/foods13091424
Academic Editors: Ilija Djekic
and Nada Smigic
Received: 30 March 2024
Revised: 15 April 2024
Accepted: 18 April 2024
Published: 6 May 2024
Copyright: © 2024 by the authors.
Licensee MDPI, Basel, Switzerland.
This article is an open access article
distributed
under
the
terms
and
conditions of the Creative Commons
Attribution (CC BY) license (https://
creativecommons.org/licenses/by/
4.0/).
foods
Article
Analysis of Actual Fitness Supplement Consumption among
Health and Fitness Enthusiasts
Paolo Renzo Espeño 1, Ardvin Kester S. Ong 1,2,*
, Josephine D. German 1
, Ma. Janice J. Gumasing 3
and Ethan S. Casas 1
1
School of Industrial Engineering and Engineering Management, Mapúa University, 658 Muralla St.,
Intramuros, Manila 1002, Philippines
2
E.T. Yuchengo Scho

In [12]:
import os

pdfs = []
for root, dirs, files in os.walk('data'):
    # print(root, dirs, files)
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))

In [22]:
pdfs

['data\\1. Analysis of Actual Fitness Supplement.pdf',
 'data\\2. High Prevalence of Supplement Intake.pdf']

In [13]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()

    docs.extend(pages)

In [23]:
docs

[Document(metadata={'source': 'data\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': 'data\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 0, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='Citation: Espeño, P.R.; Ong, A.K.S.;\nGerman, J.D.; Gumasing, M.J.J.; Casas,\nE.S. Analysis of Actual Fitness\nSupplement Consumption among\nHealth and Fitness Enthusiasts. Foods\n2024, 13, 1424. https://doi.org/\n10.3390/foods13091424\nAcademic Editors: Ilija Djekic\nand Nada Smigic\nReceived: 30 March 2024\nRevised: 15 April 2024\nAccepted: 18 April 2024\nPublished: 6 May 2024\nCopyright: © 2024 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed\nunder\nthe\nterms\nand\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommo

In [14]:
len(docs)

26

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = text_splitter.split_documents(docs)

In [16]:
len(docs), len(chunks)

(26, 110)

In [17]:
len(docs[0].page_content), len(chunks[0].page_content)

(4340, 981)

In [18]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

len(encoding.encode(docs[0].page_content)), len(encoding.encode(chunks[0].page_content))

(969, 294)

In [24]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [25]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")

single_vector = embeddings.embed_query("this is some text data")

In [26]:
len(single_vector)

768

In [27]:
index = faiss.IndexFlatL2(len(single_vector))
index.ntotal, index.d

(0, 768)

In [28]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [31]:
vector_store

In [29]:
len(chunks)

110

In [30]:
ids = vector_store.add_documents(documents=chunks)

In [32]:
ids

['c19de8e6-1a8b-469e-b624-04dfa2d21516',
 'fca90196-fadd-4b96-8c3b-82c373c21e9c',
 '056137bc-b03d-40ab-becb-0f249efe3b1f',
 '6ef602cd-5dd7-466d-aec6-3e34b9775ef6',
 '89392327-a164-44d4-91f0-0c1c03cb1871',
 'db5798a1-47c8-404f-8392-20bcbe79f52a',
 '1d632bd8-8a14-4f43-a000-8081dfefd11d',
 '24e1d22f-cd24-44b8-a8e4-19055b68e576',
 'c54feee3-6b79-4b01-a6b1-129ac076469d',
 'c20bafbf-60c0-41b0-9ab5-f92a9669b865',
 '50ab9a69-a268-4028-8058-7d673d03b572',
 '8935db04-b79f-403c-86a9-cf155ea82c0a',
 '7af61dba-7064-48db-804a-8c578ef4bbdc',
 'a5ebfcb1-73eb-43cf-b739-98e2c18e670b',
 '09a61415-9c3a-41bf-807a-a23c3c8b589d',
 'fd5415b4-5a3b-4c30-845a-b8fed7917a39',
 'b43c2163-9047-4582-a56d-73ba29e403b4',
 'a50a6933-f7e9-4d8d-8db8-2331bc4d6e9e',
 'c426e937-bf98-46f8-a38c-a7681c2d2ad2',
 'c394d418-c396-47c8-9458-6b7cc7bad2d5',
 'fb2e46ee-e8e8-4299-ab38-41a367d0ca65',
 'eb37c982-c31e-42de-bb3f-c1ade61597db',
 '698d235c-d6b9-4f93-818b-230304123a9a',
 'aca43ab2-3f8e-47cf-ae87-e23948992cef',
 '2d3610ad-6cd0-

In [33]:
len(ids)

110

In [36]:
# db_name = "health_supplements"
# vector_store.save_local(db_name)

In [37]:
# new_vector_store = FAISS.load_local(db_name, embeddings=embeddings, allow_dangerous_deserialization=True)
# len(new_vector_store.index_to_docstore_id)

In [38]:
question = "what is used to gain muscle mass?"
docs = vector_store.search(query=question, search_type='similarity')

for doc in docs:
    print(doc.page_content)
    print("\n\n")

acids than traditional protein sources. Its numerous benefits have made it a popular choice
for snacks and drinks among consumers [3]. Another widely embraced supplement is
caffeine, which is found in many sports and food supplements. Caffeine reduces perceived
effort, minimizes fatigue and pain, and proves to be effective for endurance and high-
intensity activities, which is the choice of consumers [4].
Creatine monohydrate is another well-known supplement used to gain muscle mass
and support performance and recovery. It is known not to increase fat mass and remains
effective even when taken in recommended doses [5]. Despite its popularity in the fitness
Foods 2024, 13, 1424. https://doi.org/10.3390/foods13091424
https://www.mdpi.com/journal/foods



and strength gain among men. We detected more prevalent protein and creatine supplementation
among younger compared to older ﬁtness center users, whereas the opposite was found for vitamin
supplementation. Other authors made similar obse

In [39]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs = {'k': 3, 
                                                                          'fetch_k': 100,
                                                                          'lambda_mult': 1})

In [41]:
docs = retriever.invoke(question)
for doc in docs:
    print(doc.page_content)
    print("\n\n")

acids than traditional protein sources. Its numerous benefits have made it a popular choice
for snacks and drinks among consumers [3]. Another widely embraced supplement is
caffeine, which is found in many sports and food supplements. Caffeine reduces perceived
effort, minimizes fatigue and pain, and proves to be effective for endurance and high-
intensity activities, which is the choice of consumers [4].
Creatine monohydrate is another well-known supplement used to gain muscle mass
and support performance and recovery. It is known not to increase fat mass and remains
effective even when taken in recommended doses [5]. Despite its popularity in the fitness
Foods 2024, 13, 1424. https://doi.org/10.3390/foods13091424
https://www.mdpi.com/journal/foods



and strength gain among men. We detected more prevalent protein and creatine supplementation
among younger compared to older ﬁtness center users, whereas the opposite was found for vitamin
supplementation. Other authors made similar obse

In [42]:
question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
# question = "what are the benefits of BCAA supplements?"
docs = retriever.invoke(question)

In [43]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_ollama import ChatOllama

In [44]:
model = ChatOllama(model="llama3.2:1b", base_url="http://localhost:11434")

model.invoke("hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-01-06T20:56:48.1829329Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9563016500, 'load_duration': 4375453400, 'prompt_eval_count': 26, 'prompt_eval_duration': 3882000000, 'eval_count': 8, 'eval_duration': 1151000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-8290ea81-d4b8-4eaf-87f7-e11694021caf-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [45]:
prompt = hub.pull("rlm/rag-prompt")

In [51]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [52]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# print(format_docs(docs))

In [53]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [54]:
question = "what is used to increase mass of the Earth?"

output = rag_chain.invoke(question)
print(output)

Here are the answers to the question "What is used to increase mass of the Earth?" in bullet points:

• None - There is no mention of any specific supplement or substance being used to increase the mass of the Earth in the given context.
• No direct answer, but it mentions that effort and movement are affected by automatic and controlled precursors. This implies that external factors can influence individual behavior and physical activity levels.


In [56]:
# question = "what is used to gain muscle mass?"
# question = "what is used to reduce weight?"
question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
# question = "what are the benefits of BCAA supplements?"

#question = "what is used to increase mass of the Earth?"

output = rag_chain.invoke(question)
print(output)

Here are the side effects of supplements in bullet points based on the provided context:

• Supplements may cause negative health outcomes or performance issues when used inappropriately.
• Many supplements lack evidence for any potentially useful health or performance benefits, which can lead to misuse and harm.
• Contamination with substances like non-declared anabolic steroids, stimulants, or heavy metals is a concern.
• Unsubstantiated claims made by manufacturers can mislead athletes and expose them to harmful effects.
• The internet, physicians, and coaches/trainers are the three leading sources of information on potential risks and side effects of supplements.
• Consumers often seek out information from these sources if they believe there are risks and benefits to be communicated on the label.
• Knowledge is a significant variable in consumers' attitudes, beliefs, and intentions toward supplement use.
• As knowledge grows, so does understanding, belief, and intention to use supp